In [42]:
import numpy as np
import json
import warnings

import h5py
from keras.models import model_from_json
from keras import backend as K

from matplotlib import pyplot as plt

warnings.filterwarnings("ignore")

size_title = 18
size_label = 14
n_pred = 2

path_data_dict = "data/generated_files/data_dict.txt"
path_inverted_wt = "data/generated_files/inverted_weights.txt"
path_usage_wt = "data/generated_files/usage_prediction.txt"
path_class_wt = "data/generated_files/class_weights.txt"
path_test_data = "data/generated_files/test_data.txt"
model_path = "data/generated_files/trained_model.hdf5"

def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data

def create_model(model_path):
    trained_model = h5py.File(model_path, 'r')
    model_config = json.loads(trained_model.get('model_config').value)
    loaded_model = model_from_json(model_config)
    dictionary = json.loads(trained_model.get('data_dictionary').value)
    compatibile_tools = json.loads(trained_model.get('compatible_tools').value)
    reverse_dictionary = dict((v, k) for k, v in dictionary.items())
    model_weights = list()
    weight_ctr = 0
    while True:
        try:
            d_key = "weight_" + str(weight_ctr)
            weights = trained_model.get(d_key).value
            model_weights.append(weights)
            weight_ctr += 1
        except Exception as exception:
            break
    # set the model weights
    loaded_model.set_weights(model_weights)
    return loaded_model, dictionary, reverse_dictionary, compatibile_tools

model, dictionary, reverse_dictionary, compatibile_tools = create_model(model_path)


In [ ]:
def verify_model(model, tool_sequence, labels, dictionary, reverse_dictionary, compatible_tools, topk=3, max_seq_len=25):
    
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[int(tl_seq[-1])]
    last_compatible_tools = compatible_tools[last_tool_name]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[-(idx + 1)] = int(tool_id)
        sample_reshaped = np.reshape(sample, (1, max_seq_len))

        # predict next tools for a test path
        prediction = model.predict(sample_reshaped, verbose=0)
        prediction = np.reshape(prediction, (prediction.shape[1],))
        prediction_pos = np.argsort(prediction, axis=-1)

        # get topk prediction
        topk_prediction_pos = prediction_pos[-topk:]
        topk_prediction_val = [np.round(prediction[pos] * 100, 2) for pos in topk_prediction_pos]
        
        print(topk_prediction_pos, topk_prediction_val)

        # read tool names using reverse dictionary
        pred_tool_ids = [reverse_dictionary[int(tool_pos)] for tool_pos in topk_prediction_pos]
        actual_next_tool_ids = list(set(pred_tool_ids).intersection(set(last_compatible_tools.split(","))))
        
        print(pred_tool_ids, actual_next_tool_ids)

        pred_tool_ids_sorted = dict()
        for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
            tool_name = reverse_dictionary[int(tool_pos)]
            #if tool_name in actual_next_tool_ids:
            pred_tool_ids_sorted[tool_name] = tool_pred_val
        print(pred_tool_ids_sorted)
        print("======================================")
        # find false positives
        #false_positives = [ tool_name for tool_name in top_predicted_next_tool_names if tool_name not in actual_next_tool_names ]
        #absolute_precision = 1 - ( len( false_positives ) / float( len( actual_classes_pos ) ) )
        #ave_abs_precision.append(absolute_precision)
    #mean_precision = np.mean(ave_abs_precision)
    #print("Absolute precision on test data using current model is: %0.6f" % mean_precision)
    #return mean_precision

def get_predictions(model, dictionary, reverse_dictionary, compatibile_tools):
    t_data = read_file(path_test_data)
    for ph, cl in t_data.items():
        verify_model(model, ph, cl, dictionary, reverse_dictionary, compatibile_tools)

get_predictions(model, dictionary, reverse_dictionary, compatibile_tools)

[520 553 841] [7.64, 9.09, 72.55]
['rna_star', 'bowtie2', 'fastqc'] ['rna_star', 'fastqc']
{'rna_star': 7.64, 'bowtie2': 9.09, 'fastqc': 72.55}
[439 797 841] [2.52, 4.07, 38.7]
['join1', 'multiqc', 'fastqc'] ['join1', 'fastqc']
{'join1': 2.52, 'multiqc': 4.07, 'fastqc': 38.7}
[192 813 841] [7.31, 8.31, 13.96]
['Cut1', 'tp_easyjoin_tool', 'fastqc'] ['Cut1', 'fastqc']
{'Cut1': 7.31, 'tp_easyjoin_tool': 8.31, 'fastqc': 13.96}
[439 841 192] [7.8, 8.1, 12.94]
['join1', 'fastqc', 'Cut1'] ['Cut1', 'join1', 'fastqc']
{'join1': 7.8, 'fastqc': 8.1, 'Cut1': 12.94}
[192  24 439] [2.58, 3.04, 4.48]
['Cut1', 'Add_a_column1', 'join1'] ['Cut1', 'join1']
{'Cut1': 2.58, 'Add_a_column1': 3.04, 'join1': 4.48}
[192 261 563] [4.02, 6.43, 13.3]
['Cut1', 'tp_sort_header_tool', 'tp_cat'] ['Cut1', 'tp_sort_header_tool']
{'Cut1': 4.02, 'tp_sort_header_tool': 6.43, 'tp_cat': 13.3}
[841 439 563] [3.46, 4.58, 15.99]
['fastqc', 'join1', 'tp_cat'] ['join1', 'fastqc']
{'fastqc': 3.46, 'join1': 4.58, 'tp_cat': 15.99}
[

[192 813 841] [7.31, 8.31, 13.96]
['Cut1', 'tp_easyjoin_tool', 'fastqc'] ['Cut1', 'tp_easyjoin_tool']
{'Cut1': 7.31, 'tp_easyjoin_tool': 8.31, 'fastqc': 13.96}
[813 261 192] [14.6, 24.63, 28.53]
['tp_easyjoin_tool', 'tp_sort_header_tool', 'Cut1'] ['Cut1', 'tp_easyjoin_tool']
{'tp_easyjoin_tool': 14.6, 'tp_sort_header_tool': 24.63, 'Cut1': 28.53}
[204 841 813] [1.92, 4.08, 18.14]
['Convert characters1', 'fastqc', 'tp_easyjoin_tool'] ['tp_easyjoin_tool']
{'Convert characters1': 1.92, 'fastqc': 4.08, 'tp_easyjoin_tool': 18.14}
[439 204 813] [3.82, 6.8, 29.2]
['join1', 'Convert characters1', 'tp_easyjoin_tool'] ['join1', 'tp_easyjoin_tool']
{'join1': 3.82, 'Convert characters1': 6.8, 'tp_easyjoin_tool': 29.2}
[775 841 813] [0.57, 6.45, 8.16]
['collapse_dataset', 'fastqc', 'tp_easyjoin_tool'] ['tp_easyjoin_tool']
{'collapse_dataset': 0.57, 'fastqc': 6.45, 'tp_easyjoin_tool': 8.16}
[816 841 813] [1.94, 4.03, 6.3]
['Grouping1', 'fastqc', 'tp_easyjoin_tool'] ['tp_easyjoin_tool']
{'Grouping1': 

[813 841 439] [7.63, 12.0, 12.96]
['tp_easyjoin_tool', 'fastqc', 'join1'] ['join1', 'tp_easyjoin_tool']
{'tp_easyjoin_tool': 7.63, 'fastqc': 12.0, 'join1': 12.96}
[439 841 813] [1.8, 2.95, 6.39]
['join1', 'fastqc', 'tp_easyjoin_tool'] ['join1', 'tp_easyjoin_tool']
{'join1': 1.8, 'fastqc': 2.95, 'tp_easyjoin_tool': 6.39}
[841 439 813] [2.94, 6.8, 14.43]
['fastqc', 'join1', 'tp_easyjoin_tool'] ['join1', 'tp_easyjoin_tool']
{'fastqc': 2.94, 'join1': 6.8, 'tp_easyjoin_tool': 14.43}
[841 813 439] [1.19, 4.71, 8.52]
['fastqc', 'tp_easyjoin_tool', 'join1'] ['join1', 'tp_easyjoin_tool']
{'fastqc': 1.19, 'tp_easyjoin_tool': 4.71, 'join1': 8.52}
[204 439 813] [0.49, 1.99, 8.29]
['Convert characters1', 'join1', 'tp_easyjoin_tool'] ['join1', 'tp_easyjoin_tool']
{'Convert characters1': 0.49, 'join1': 1.99, 'tp_easyjoin_tool': 8.29}
[204 439 813] [1.39, 3.54, 15.39]
['Convert characters1', 'join1', 'tp_easyjoin_tool'] ['join1', 'tp_easyjoin_tool']
{'Convert characters1': 1.39, 'join1': 3.54, 'tp_eas

[192 816 439] [2.53, 3.17, 62.5]
['Cut1', 'Grouping1', 'join1'] ['join1', 'Grouping1']
{'Cut1': 2.53, 'Grouping1': 3.17, 'join1': 62.5}
[457 816 439] [1.58, 3.47, 49.66]
['Remove beginning1', 'Grouping1', 'join1'] ['Remove beginning1', 'join1', 'Grouping1']
{'Remove beginning1': 1.58, 'Grouping1': 3.47, 'join1': 49.66}
[775 813 439] [3.27, 4.19, 54.58]
['collapse_dataset', 'tp_easyjoin_tool', 'join1'] ['collapse_dataset', 'join1', 'tp_easyjoin_tool']
{'collapse_dataset': 3.27, 'tp_easyjoin_tool': 4.19, 'join1': 54.58}
[775 841 439] [1.92, 2.08, 56.04]
['collapse_dataset', 'fastqc', 'join1'] ['collapse_dataset', 'join1']
{'collapse_dataset': 1.92, 'fastqc': 2.08, 'join1': 56.04}
[813 816 439] [3.05, 9.05, 59.89]
['tp_easyjoin_tool', 'Grouping1', 'join1'] ['join1', 'tp_easyjoin_tool', 'Grouping1']
{'tp_easyjoin_tool': 3.05, 'Grouping1': 9.05, 'join1': 59.89}
[520 553 841] [7.64, 9.09, 72.55]
['rna_star', 'bowtie2', 'fastqc'] ['rna_star', 'fastqc']
{'rna_star': 7.64, 'bowtie2': 9.09, 'fas

[816 192 439] [6.76, 13.77, 61.72]
['Grouping1', 'Cut1', 'join1'] ['Cut1', 'Grouping1']
{'Grouping1': 6.76, 'Cut1': 13.77, 'join1': 61.72}
[813 192 439] [4.37, 11.35, 57.89]
['tp_easyjoin_tool', 'Cut1', 'join1'] ['Cut1']
{'tp_easyjoin_tool': 4.37, 'Cut1': 11.35, 'join1': 57.89}
[813 192 439] [1.51, 6.46, 33.6]
['tp_easyjoin_tool', 'Cut1', 'join1'] ['Cut1']
{'tp_easyjoin_tool': 1.51, 'Cut1': 6.46, 'join1': 33.6}
[192 813 439] [4.12, 16.21, 51.54]
['Cut1', 'tp_easyjoin_tool', 'join1'] ['Cut1']
{'Cut1': 4.12, 'tp_easyjoin_tool': 16.21, 'join1': 51.54}
[192 813 439] [2.73, 15.66, 18.09]
['Cut1', 'tp_easyjoin_tool', 'join1'] ['Cut1']
{'Cut1': 2.73, 'tp_easyjoin_tool': 15.66, 'join1': 18.09}
[192 813 439] [10.96, 13.69, 27.87]
['Cut1', 'tp_easyjoin_tool', 'join1'] ['Cut1']
{'Cut1': 10.96, 'tp_easyjoin_tool': 13.69, 'join1': 27.87}
[816 192 439] [8.53, 45.18, 84.61]
['Grouping1', 'Cut1', 'join1'] ['Cut1', 'Grouping1']
{'Grouping1': 8.53, 'Cut1': 45.18, 'join1': 84.61}
[520 553 841] [7.64, 9.0

...

[192 204 439] [1.34, 5.49, 54.81]
['Cut1', 'Convert characters1', 'join1'] ['Cut1', 'Convert characters1', 'join1']
{'Cut1': 1.34, 'Convert characters1': 5.49, 'join1': 54.81}
[520 553 841] [7.64, 9.09, 72.55]
['rna_star', 'bowtie2', 'fastqc'] []
{'rna_star': 7.64, 'bowtie2': 9.09, 'fastqc': 72.55}
[439 797 841] [1.69, 1.71, 35.98]
['join1', 'multiqc', 'fastqc'] ['join1']
{'join1': 1.69, 'multiqc': 1.71, 'fastqc': 35.98}
[813 439 841] [5.92, 9.17, 24.87]
['tp_easyjoin_tool', 'join1', 'fastqc'] ['join1', 'tp_easyjoin_tool']
{'tp_easyjoin_tool': 5.92, 'join1': 9.17, 'fastqc': 24.87}
[816 813 439] [3.63, 4.73, 14.94]
['Grouping1', 'tp_easyjoin_tool', 'join1'] ['join1', 'tp_easyjoin_tool']
{'Grouping1': 3.63, 'tp_easyjoin_tool': 4.73, 'join1': 14.94}
[204 816 439] [10.35, 19.49, 72.6]
['Convert characters1', 'Grouping1', 'join1'] ['join1']
{'Convert characters1': 10.35, 'Grouping1': 19.49, 'join1': 72.6}
[204 816 439] [12.79, 24.96, 90.43]
['Convert characters1', 'Grouping1', 'join1'] ['jo

[439 797 841] [2.52, 4.07, 38.7]
['join1', 'multiqc', 'fastqc'] ['join1']
{'join1': 2.52, 'multiqc': 4.07, 'fastqc': 38.7}
[192 813 841] [7.31, 8.31, 13.96]
['Cut1', 'tp_easyjoin_tool', 'fastqc'] ['Cut1', 'tp_easyjoin_tool']
{'Cut1': 7.31, 'tp_easyjoin_tool': 8.31, 'fastqc': 13.96}
[520 192  24] [4.39, 7.85, 10.23]
['rna_star', 'Cut1', 'Add_a_column1'] ['Cut1']
{'rna_star': 4.39, 'Cut1': 7.85, 'Add_a_column1': 10.23}
[816 439  24] [9.64, 13.4, 14.99]
['Grouping1', 'join1', 'Add_a_column1'] ['join1', 'Grouping1']
{'Grouping1': 9.64, 'join1': 13.4, 'Add_a_column1': 14.99}
[439  24 192] [3.53, 4.52, 5.9]
['join1', 'Add_a_column1', 'Cut1'] ['Cut1', 'join1']
{'join1': 3.53, 'Add_a_column1': 4.52, 'Cut1': 5.9}
[952 439 192] [5.79, 7.62, 11.62]
['melt', 'join1', 'Cut1'] ['Cut1', 'join1']
{'melt': 5.79, 'join1': 7.62, 'Cut1': 11.62}
[ 24 192 439] [2.7, 4.84, 8.04]
['Add_a_column1', 'Cut1', 'join1'] ['Cut1', 'join1']
{'Add_a_column1': 2.7, 'Cut1': 4.84, 'join1': 8.04}
[816 192 439] [2.64, 7.3, 

[841 816 813] [0.51, 0.63, 46.82]
['fastqc', 'Grouping1', 'tp_easyjoin_tool'] ['tp_easyjoin_tool', 'Grouping1']
{'fastqc': 0.51, 'Grouping1': 0.63, 'tp_easyjoin_tool': 46.82}
[520 553 841] [7.64, 9.09, 72.55]
['rna_star', 'bowtie2', 'fastqc'] []
{'rna_star': 7.64, 'bowtie2': 9.09, 'fastqc': 72.55}
[439 797 841] [1.69, 1.71, 35.98]
['join1', 'multiqc', 'fastqc'] ['join1']
{'join1': 1.69, 'multiqc': 1.71, 'fastqc': 35.98}
[813 439 841] [4.71, 6.52, 9.11]
['tp_easyjoin_tool', 'join1', 'fastqc'] ['join1', 'tp_easyjoin_tool']
{'tp_easyjoin_tool': 4.71, 'join1': 6.52, 'fastqc': 9.11}
[813 841 439] [7.53, 8.72, 25.84]
['tp_easyjoin_tool', 'fastqc', 'join1'] ['join1', 'tp_easyjoin_tool']
{'tp_easyjoin_tool': 7.53, 'fastqc': 8.72, 'join1': 25.84}
[841 813 439] [5.48, 7.94, 45.46]
['fastqc', 'tp_easyjoin_tool', 'join1'] ['join1', 'tp_easyjoin_tool']
{'fastqc': 5.48, 'tp_easyjoin_tool': 7.94, 'join1': 45.46}
[841 813 439] [6.83, 8.57, 18.93]
['fastqc', 'tp_easyjoin_tool', 'join1'] ['join1', 'tp_e

[520 553 841] [7.64, 9.09, 72.55]
['rna_star', 'bowtie2', 'fastqc'] []
{'rna_star': 7.64, 'bowtie2': 9.09, 'fastqc': 72.55}
[439 797 841] [1.69, 1.71, 35.98]
['join1', 'multiqc', 'fastqc'] ['join1']
{'join1': 1.69, 'multiqc': 1.71, 'fastqc': 35.98}
[813 439 841] [5.92, 9.17, 24.87]
['tp_easyjoin_tool', 'join1', 'fastqc'] ['join1', 'tp_easyjoin_tool']
{'tp_easyjoin_tool': 5.92, 'join1': 9.17, 'fastqc': 24.87}
[816 813 439] [3.63, 4.73, 14.94]
['Grouping1', 'tp_easyjoin_tool', 'join1'] ['join1', 'tp_easyjoin_tool', 'Grouping1']
{'Grouping1': 3.63, 'tp_easyjoin_tool': 4.73, 'join1': 14.94}
[192 816 439] [7.38, 18.99, 77.94]
['Cut1', 'Grouping1', 'join1'] ['join1', 'Grouping1']
{'Cut1': 7.38, 'Grouping1': 18.99, 'join1': 77.94}
[204 816 439] [2.39, 5.23, 56.17]
['Convert characters1', 'Grouping1', 'join1'] ['Convert characters1', 'join1', 'Grouping1']
{'Convert characters1': 2.39, 'Grouping1': 5.23, 'join1': 56.17}
[816 204 439] [4.72, 4.84, 35.27]
['Grouping1', 'Convert characters1', 'joi

[192 204 439] [6.07, 6.49, 90.22]
['Cut1', 'Convert characters1', 'join1'] ['Cut1', 'join1']
{'Cut1': 6.07, 'Convert characters1': 6.49, 'join1': 90.22}
[520 553 841] [7.64, 9.09, 72.55]
['rna_star', 'bowtie2', 'fastqc'] []
{'rna_star': 7.64, 'bowtie2': 9.09, 'fastqc': 72.55}
[439 797 841] [2.52, 4.07, 38.7]
['join1', 'multiqc', 'fastqc'] ['join1']
{'join1': 2.52, 'multiqc': 4.07, 'fastqc': 38.7}
[192 813 841] [7.31, 8.31, 13.96]
['Cut1', 'tp_easyjoin_tool', 'fastqc'] ['Cut1', 'tp_easyjoin_tool']
{'Cut1': 7.31, 'tp_easyjoin_tool': 8.31, 'fastqc': 13.96}
[520 192  24] [4.39, 7.85, 10.23]
['rna_star', 'Cut1', 'Add_a_column1'] ['Cut1']
{'rna_star': 4.39, 'Cut1': 7.85, 'Add_a_column1': 10.23}
[816 439  24] [9.64, 13.4, 14.99]
['Grouping1', 'join1', 'Add_a_column1'] ['join1']
{'Grouping1': 9.64, 'join1': 13.4, 'Add_a_column1': 14.99}
[439  24 192] [3.53, 4.52, 5.9]
['join1', 'Add_a_column1', 'Cut1'] ['Cut1', 'join1']
{'join1': 3.53, 'Add_a_column1': 4.52, 'Cut1': 5.9}
[813 816  24] [1.25, 2